In [ ]:
import os
import pickle
import random
from collections import defaultdict

import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import tensorflow as tf
from matplotlib import pyplot as plt
from keras import layers, models, optimizers

In [ ]:
MAIN_FOLDER_PATH='C:\\Users\\renat\\Projetos\\Harmful Brain Activity Classification'

TARGETS = ['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']

CHAINS = {
    'LL' : [("Fp1","F7"),("F7","T3"),("T3","T5"),("T5","O1")],
    'RL' : [("Fp2","F8"),("F8","T4"),("T4","T6"),("T6","O2")],
    'LP' : [("Fp1","F3"),("F3","C3"),("C3","P3"),("P3","O1")],
    'RP' : [("Fp2","F4"),("F4","C4"),("C4","P4"),("P4","O2")]
}

EEG_COL_NAMES=['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz',
       'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2', 'EKG']

EEG_DDP_COL_NAMES=['Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1', 'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O2',
               'Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'Fp2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'EKG']

K = 1.5 # parâmetro para detecção de outliers (e.g limInf = q1 - k*IQR)

In [ ]:
def get_sample(row,metadata,window=True):
    
    eeg_id = str(metadata.loc[row,'eeg_id'])
    eeg_offset = int(metadata.loc[row,'eeg_label_offset_seconds'])
    
    spect_id = str(metadata.loc[row,'spectrogram_id'])
    spect_offset = int(metadata.loc[row,'spectrogram_label_offset_seconds'])
    
    eeg = pd.read_parquet(MAIN_FOLDER_PATH+f'\\train_eegs\\{eeg_id}.parquet')
    spect = pd.read_parquet(MAIN_FOLDER_PATH+f'\\train_spectrograms\\{spect_id}.parquet')
    
    # fill nans with the mean
    
    eeg=eeg.fillna(eeg.mean())
    spect=spect.fillna(spect.mean())
    
    if window:        
        length=10
        eeg = eeg.iloc[(eeg_offset+20)*200:(eeg_offset+30)*200]
        spect = spect.loc[(spect.time>=spect_offset+295)&(spect.time<spect_offset+305)]
        
    else:
        length=50
        eeg = eeg.iloc[eeg_offset*200:(eeg_offset+50)*200]
        spect = spect.loc[(spect.time>=spect_offset)&(spect.time<spect_offset+600)]
        
    label = metadata.loc[row,TARGETS].values
        
    return eeg, spect, label

def load_data(data,window=True):
    
    eeg_tensor=[]
    sp_tensor=[]
    labels=[]
    
    for row_index in range(data.shape[0]):

        try:
            eeg, spect, label = get_sample(row_index,metadata=data,window=window)       
        except:
            print(f'There was a problem with row {row_index}!')
        else:
            eeg_tensor.append(eeg.values)
            sp_tensor.append(spect.iloc[:,1:].values)
            labels.append(label)
    
    return np.stack(eeg_tensor), np.stack(sp_tensor), np.array(labels)

In [ ]:
# Retorna um mapa indicando quais colunas, em cada amostra, possuem valores aberrantes 

def tell_liers(eegs, return_lims=False):

    liers=pd.DataFrame({col:[0]*eegs.shape[0] for col in EEG_COL_NAMES})
    
    limites_dic=defaultdict(list)
    
    for col in range(eegs.shape[2]):

        limites = {'lim_inf':[],'lim_sup':[]}

        for index, sample in enumerate(eegs):

            arr=sample[:,col]

            #if np.std(arr)==0: liers.loc[index,EEG_COL_NAMES[col]]=1

            q1, q2, q3 = np.percentile(arr, [25, 50, 75])
            iqr = q3 - q1

            lim_sup = q3 + K * iqr
            lim_inf = q1 - K * iqr

            limites['lim_inf'].append(lim_inf)
            limites['lim_sup'].append(lim_sup)

        # calcular os LIM_INF e LIM_SUP dos pools de lim_infs e lim_sups       

        pool_infs = np.array(limites['lim_inf'])
        pool_sups = np.array(limites['lim_sup'])
        
        limites_dic[col]+=[pool_infs,pool_sups]
        
        q_infs = np.percentile(pool_infs, [25, 50, 75])
        q_sups = np.percentile(pool_sups, [25, 50, 75])

        iqr_infs = q_infs[2] - q_infs[0] # IQR do conjunto de limites inferiores
        iqr_sups = q_sups[2] - q_sups[0] # IQR do conjunto de limites superiores

        LIM_INF = q_infs[0] - K * iqr_infs
        LIM_SUP = q_sups[2] + K * iqr_sups

        # agora, considero outlier toda amostra cujo lim_inf seja menor que LIM_INF e toda amostra cujo lim_sup seja maior que LIM_SUP

        for index, lim in enumerate(zip(limites['lim_inf'],limites['lim_sup'])):
            if lim[0]<LIM_INF or lim[1]>LIM_SUP:
                liers.loc[index,EEG_COL_NAMES[col]]=1
                
    if return_lims==True:
        
        return liers, limites_dic
    
    elif return_lims==False:
        
        return liers
    

def big_liers(liers, threshold=10):
    
    # retorna um dataframe contendo as amostras que devem ser removidas
    
    big_liers=[]
    
    for index, row in liers.iterrows():
        
        if row.sum()>threshold:
            big_liers.append(index)
            
    print(f'Número de big liers: {len(big_liers)}')        
    print(f'Proporção de outliers: {len(big_liers)/liers.shape[0]}')
    
    return big_liers

## Load metadata, select non-overlaping samples and transform labels into probability distributions

In [ ]:
train = pd.read_csv(MAIN_FOLDER_PATH+'\\train.csv')

df=train.groupby('eeg_id')[['spectrogram_id', 'eeg_label_offset_seconds','spectrogram_label_offset_seconds', 'patient_id']].agg({
    'spectrogram_id':'first', 'eeg_label_offset_seconds':'min', 'spectrogram_label_offset_seconds':'min', 'patient_id':'first'})


# normalizing labels

tmp = train.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    df[t] = tmp[t].values
    
y_data = df[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
df[TARGETS] = y_data

tmp = train.groupby('eeg_id')[['expert_consensus']].agg('first')
df['target'] = tmp

df = df.reset_index()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,5))

sns.histplot(data=train.expert_consensus, ax=ax[0])
ax[0].set_title("'train' target distribution")

sns.histplot(data=df.target, ax=ax[1])
ax[1].set_title("'df' target distribution")

## Load data from files

In [ ]:
def load_grouped(df, column='target',window=True):
    
    output = defaultdict(list)
    
    groups=df.groupby(column)
    
    for group_name, group_data in groups:
        
        print(f'Loading group {group_name}.')
        
        eegs, spects, labels = load_data(group_data.reset_index().drop('index', axis=1),window=window)
        
        output[group_name]+=[eegs, spects, labels]
        
    return output

## Load data from pickle

In [ ]:
%%time
with open('data_dict.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data.keys()

## Split data into train set and test set

Estou fazendo o split antes da remoção dos outliers. Isso significa que pode haver outliers no conjunto de teste. Como não sabemos se os dados de teste da competição possuem colunas absurdas, acredito que vale a pena tentar treinar o modelo dos dois jeitos: removendo os outliers antes e depois do split. Por ora seguirei com a segunda opção.

In [ ]:
train_ratio = 0.80

In [ ]:
train_data = {}
test_data = {}

for label, dados in data.items():
    
    eegs = dados[0]
    specs = dados[1]
    labels = dados[2]
    
    idx = [i for i in range(eegs.shape[0])]
    random.shuffle(idx)
    
    train_idx = idx[0:int(train_ratio*len(idx))]
    test_idx = idx[int(train_ratio*len(idx)):]
    
    train_data[label] = [eegs[train_idx], specs[train_idx], labels[train_idx]]
    test_data[label] = [eegs[test_idx], specs[test_idx], labels[test_idx]]

In [ ]:
def plot_target_distribution(data, ax):
    
    labels = []
    freqs = []
    
    for label, dados in data.items():
        
        labels.append(label)
        freqs.append(dados[0].shape[0])
    
    ser = []
    
    for label, freq in zip(labels,freqs):
        ser+=[label]*freq
    
    ser = pd.Series(ser)
    
    sns.histplot(ser, ax=ax, kde=True)

In [ ]:
fig, ax = plt.subplots(3,figsize=(20,20))

titles=['data','train data', 'test data']

for i, dados in enumerate([data, train_data, test_data]):
    
    plot_target_distribution(dados,ax[i])
    ax[i].set_title(titles[i])

## Out, liers!

In [ ]:
THRESHOLD = 10

In [ ]:
liers=tell_liers(train_data['Seizure'][0])

In [ ]:
bigliers=big_liers(liers, threshold=7)

In [ ]:
liers=tell_liers(train_data['Other'][0])

In [ ]:
bigliers=big_liers(liers, threshold=7)

In [ ]:
def remove_outliers(data,threshold=THRESHOLD):
    
    output={}
    
    for label, dados in data.items():
        
        print(label)
        
        eegs = dados[0]
        specs = dados[1]
        labels = dados[2]
        
        liers = tell_liers(eegs)
        idx_to_drop = big_liers(liers, threshold=threshold)
        
        output[label] = [np.delete(eegs, idx_to_drop, axis=0),np.delete(specs, idx_to_drop, axis=0),np.delete(labels, idx_to_drop, axis=0)]
        
        print('_________________________________________________________________________')
        
    return output

In [ ]:
train_data = remove_outliers(train_data)

In [ ]:
fig, ax = plt.subplots()

plot_target_distribution(train_data,ax=ax)

## Prune "Other" in train and test set

Amostras classificadas como "Other" são muito mais frequentes. Como não queremos que o modelo fique especialista em prever "Other" e esqueça de suas outras responsabilidades, irei remover amostras dessa classe até que os dados estejam mais balanceados.  

In [ ]:
# calcular a média das quantidades de amostras de cada classe, exceto Other. 

lengths=[]

for label, dados in train_data.items():
    
    print(label)
    
    eegs=dados[0]
    specs=dados[1]
    labels=dados[2]
    
    length=eegs.shape[0]
    
    print(f'{length} samples')
    print('_________________________________________________________________________')
    
    if label != 'Other':
        lengths.append(length)
    
media_train = sum(lengths)/len(lengths)

print(f'Media (tirando "Other"): {media_train}')

In [ ]:
# calcular a média das quantidades de amostras de cada classe, exceto Other. 

lengths=[]

for label, dados in test_data.items():
    
    print(label)
    
    eegs=dados[0]
    specs=dados[1]
    labels=dados[2]
    
    length=eegs.shape[0]
    
    print(f'{length} samples')
    print('_________________________________________________________________________')
    
    if label != 'Other':
        lengths.append(length)
    
media_test = sum(lengths)/len(lengths)

print(f'Media (tirando "Other"): {media_test}')

In [ ]:
train_data['Other']=[train_data['Other'][0][0:int(media_train)],train_data['Other'][1][0:int(media_train)],train_data['Other'][2][0:int(media_train)]]

In [ ]:
test_data['Other']=[test_data['Other'][0][0:int(media_test)],test_data['Other'][1][0:int(media_test)],test_data['Other'][2][0:int(media_test)]]

In [ ]:
fig, ax = plt.subplots(2)

plot_target_distribution(train_data, ax=ax[0])
ax[0].set_title('Pruned train data')

plot_target_distribution(test_data, ax=ax[1])
ax[1].set_title('Pruned test data')

fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0, hspace=0.5)

## Data augmentation

Os dados ainda estão desbalanceados. Sugiro um aumento de amostras da classe LRDA, por exemplo. 
No entanto, vou passar esta etapa por enquanto. Depois testamos diferentes abordagens de data augmentation.

## Handle columns with std=0

Há colunas com std=0. Aceito sugestões de como tratar esses casos. Pensei em clusterizar as amostras por similaridade entre os vetores label normalizados e selecionar uma amostra representativa do cluster para "doar" cópias de suas colunas para outras amostras do mesmo cluster. Podemos inclusive aplicar uma softmax com temperatura elevada nas labels antes de clusterizar.

## Concatenate classes

In [ ]:
train_eegs = np.concatenate([dados[0] for label, dados in train_data.items()])
train_specs = np.concatenate([dados[1] for label, dados in train_data.items()])
train_labels = np.concatenate([dados[2] for label, dados in train_data.items()])

In [ ]:
test_eegs = np.concatenate([dados[0] for label, dados in test_data.items()])
test_specs = np.concatenate([dados[1] for label, dados in test_data.items()])
test_labels = np.concatenate([dados[2] for label, dados in test_data.items()])

In [ ]:
train_eegs.shape[0]

In [ ]:
test_eegs.shape[0]

## Denoise EEGs

Não implementei esta etapa, mas é algo que vale a pena considerarmos. 

Sugestão abaixo. Peguei desse link: https://www.kaggle.com/code/cdeotte/how-to-make-spectrogram-from-eeg?scriptVersionId=159854820

In [ ]:
# DENOISE FUNCTION
# testar diferentes valores para "wavelet"

#import pywt

def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise(x, wavelet='haar', level=1):    
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * maddest(coeff[-level])

    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

    ret=pywt.waverec(coeff, wavelet, mode='per')
    
    return ret

## Convert EEGs into Spectrograms

In [ ]:
def to_df(data, columns=EEG_COL_NAMES):   
    df=pd.DataFrame(data=data, columns=columns)
    return df

In [ ]:
df_sample = to_df(train_eegs[0])

In [ ]:
df_sample

In [ ]:
NPERSEG = 400

In [ ]:
def spec(data, nperseg=NPERSEG):
    return sp.signal.spectrogram(data,fs=200,nperseg=nperseg)

In [ ]:
f, t , s = spec(df_sample['Fp1'])

In [ ]:
t

In [ ]:
s.shape

In [ ]:
int(0.7*201)

In [ ]:
f[1:141]

In [ ]:
CHAINS = {
    'LL' : [("Fp1","F7"),("F7","T3"),("T3","T5"),("T5","O1")],
    'RL' : [("Fp2","F8"),("F8","T4"),("T4","T6"),("T6","O2")],
    'LP' : [("Fp1","F3"),("F3","C3"),("C3","P3"),("P3","O1")],
    'RP' : [("Fp2","F4"),("F4","C4"),("C4","P4"),("P4","O2")]
}

In [ ]:
def magic_spec(data):
    
    freqs, t, s = spec(data['Fp1'])
    n_rows=t.shape[0]
    f_lim = int(0.7*freqs.shape[0])+1
    output = []
    
    for region in CHAINS.keys():
        
        magic_spec = pd.DataFrame({region+str(f):[0]*n_rows for f in freqs[1:f_lim]})
        
        for pair in CHAINS[region]:
            
            f, t, s1 = spec(data[pair[0]])
            f, t, s2 = spec(data[pair[1]])
            
            magic_spec = magic_spec.add(pd.DataFrame(data=s1[1:f_lim].T-s2[1:f_lim].T, columns=[region+str(freq) for freq in f[1:f_lim]]))
            
        output.append(magic_spec/4)
        
    return pd.concat(output, axis=1)

In [ ]:
magic_spec(df_sample)

In [ ]:
def eegs_to_spec(eegs):
    
    output=[]
    
    for eeg in eegs:
        
        spec = magic_spec(to_df(eeg)).values
        
        output.append(spec)
        
    return np.stack(output)

In [ ]:
train_eegs.shape

In [ ]:
%%time

tr_train_eegs = eegs_to_spec(train_eegs)

In [ ]:
tr_train_eegs.shape

In [ ]:
test_eegs.shape

In [ ]:
%%time

tr_test_eegs = eegs_to_spec(test_eegs)

In [ ]:
tr_test_eegs.shape

In [ ]:
train_specs.shape

In [ ]:
test_specs.shape

## Checkpoint

Temos agora:

- tr_train_eegs
- train_specs
- train_labels
- tr_test_eegs
- test_specs
- test_labels

## Merge EEG Specs with Kaggle Specs

In [ ]:
def assemble_data(tr_eegs, specs):
    
    if tr_eegs.shape[1]==specs.shape[1]: # verifica se os dois arrays têm o mesmo número de linhas
        
        output=[]
        
        for tr_eeg, spec in zip(tr_eegs, specs):
            
            output.append(np.concatenate((tr_eeg, spec), axis=1))
        
        return np.stack(output)
                          
    else:
        print('É preciso implentar o preenchimento de um dos arrays até que ambos tenham o mesmo shape.')      

In [ ]:
%%time

X_train = assemble_data(tr_train_eegs, train_specs)

In [ ]:
X_train.shape

In [ ]:
y_train = train_labels

In [ ]:
y_train.shape

In [ ]:
X_test = assemble_data(tr_test_eegs, test_specs)

In [ ]:
y_test = test_labels

## Standardization (mean=0, std=1)

In [ ]:
def normalize_data(tensor):
    
    normalized_tensor=[]
    
    for data in tensor:
        # Calculate mean and standard deviation for each column
        mean = np.mean(data, axis=0)
        std_dev = np.std(data, axis=0)
        if 0 in std_dev: std_dev+=0.0000001
        # Center the data by subtracting the mean
        centered_data = data - mean

        # Scale the data by dividing by the standard deviation
        normalized_data = centered_data / std_dev

        normalized_tensor.append(normalized_data)
        
    return np.stack(normalized_tensor)

In [ ]:
X_train = normalize_data(X_train)

In [ ]:
X_test = normalize_data(X_test)

In [ ]:
tr_data={'X_train':X_train,'y_train':y_train,'X_test':X_test,'y_test':y_test}

In [ ]:
# Specify the file path where you want to save the dictionary
file_path = 'tr_data.pkl'

# Open the file in binary write mode and save the dictionary using pickle
with open(file_path, 'wb') as f:
    pickle.dump(tr_data, f)